In [1]:
import java.util.stream.*
import java.util.concurrent.*

In [2]:
//cpu核心数
println(Runtime.getRuntime().availableProcessors())
//最大线程数
println(ForkJoinPool.getCommonPoolParallelism())
//当前线程数
println(ForkJoinPool.commonPool().poolSize)

16
15
0


## 多个任务全部完成后，打印总体时间

In [36]:
val dished = IntStream.range(0,7)
    .mapToObj({
        CompletableFuture.runAsync(
                {
                    println(it.toString()+":"+Thread.currentThread().name+":"+System.currentTimeMillis())
                    Thread.sleep(2000)
                }
            )
    })
    .toList()

val start = System.currentTimeMillis()
CompletableFuture.allOf(*dished.toTypedArray()).join()
println(System.currentTimeMillis() - start)

0:ForkJoinPool.commonPool-worker-24:1662208665500
1:ForkJoinPool.commonPool-worker-25:1662208665525
6:ForkJoinPool.commonPool-worker-30:1662208665540
5:ForkJoinPool.commonPool-worker-29:1662208665539
4:ForkJoinPool.commonPool-worker-28:1662208665538
3:ForkJoinPool.commonPool-worker-27:1662208665533
2:ForkJoinPool.commonPool-worker-26:1662208665530
2120


## 异步执行一个任务（不阻塞笔记本）

In [32]:
val log = java.util.StringJoiner("\n")
CompletableFuture.runAsync({
    log.add("开始睡眠")
    Thread.sleep(5000)
    log.add("结束睡眠")
}).whenCompleteAsync({a,b->
    println(log.toString())
})

java.util.concurrent.CompletableFuture@1ddd3478[Not completed]

In [33]:
log

开始睡眠
结束睡眠

## 异步执行一个任务，并等待输出（阻塞笔记本）

In [31]:
CompletableFuture.runAsync({
    println("开始睡眠")
    Thread.sleep(5000)
    println("结束睡眠")
}).whenCompleteAsync({a,b->
    println(a)
    println(b)
}).join()

开始睡眠
结束睡眠
null
null


null

In [13]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")
import kotlinx.coroutines.*

In [21]:
println("===")
runBlocking {
    launch { doWorld() }
    println("Hello")
}

suspend fun doWorld() {
    delay(1000L)
    println("World!")
}
println("===")

===
Hello
World!
===


## 直接开启一个线程执行任务，不阻塞笔记本

In [21]:
println(Thread.currentThread().name)
val log = java.util.StringJoiner("\n")
val t = java.lang.Thread({
    log.add("hello "+Thread.currentThread().name)
    Thread.sleep(10000)
    log.add("world "+Thread.currentThread().name)
})
t.start()
println(t.state)

Thread-99
RUNNABLE


In [22]:
println(log)
println(t.state)
println("======")
Thread.sleep(10000)
println(log)
println(t.state)

hello Thread-100
TIMED_WAITING
hello Thread-100
world Thread-100
TERMINATED


## 从【无】开始异步接替

In [33]:
CompletableFuture.completedFuture(null)
.thenRunAsync({
    Thread.sleep(1000)
})
.thenRunAsync({
    Thread.sleep(1000)
})
.join()

null

## 如果开多个线程去跑任务，怎么align所有线程状态？